### ROT: Detecting and Forecasting the Occlusion Events between the Sun and Clouds in Realtime

In [ ]:
import os
import queue
import threading

In [ ]:
%matplotlib inline

import sys
import pandas as pd
import numpy as np
import math

import cv2
import glob

import io
from PIL import Image

import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from rot_helper.thread_functions import sky_images_generator
from rot_helper.ssd_detection import detection_data, post_process_detection_data

from rot_helper.lstm_forecast import forecast_inferrence, return_sequence_ssd_detection_data
from rot_helper.lstm_forecast import post_process_ssd_detection_data_for_forecasting, local_padding_series
from rot_helper.lstm_forecast import n_windowed_forecast_inferrence, forecast_report_on_occlusions_predictions

In [ ]:
from sklearn.metrics import mean_squared_error

#### Sky Camera Files for Images Captured on the <2017_07_15> Day.

In [ ]:
sky_camera_images_dir = '../experiments/experiment1/2017_07_15'
sky_camera_images_files = os.listdir(sky_camera_images_dir)
sky_camera_images_files.sort() # Sorted
sky_camera_images_files[:20]

#### Trained ML models hyper parameters

In [ ]:
seq_num             = 50
num_bboxes          = 5
max_bboxes_vec_size = 6 * num_bboxes # About 6 points data for each of the 5 bboxes per an image.

In [ ]:
forecasting_model_name = '../exported_ml_models/lstm_forecasting_model/toesc_forecasting_keras_lambda_model.h5' # A simple forecasting model to predict the next bboxes

#### Data Structures to hold intermediary data 

In [ ]:
# To capture inbound images from the ground sky camera in realtime (i.e., a new image per second).
queue_of_inbound_sky_images = queue.Queue()

In [ ]:
# To capture a sequence of images of length = seq_num (i.e., 50) to be used to forecast the next image detection data.
queue_of_images_sequence_for_forecast_task = queue.Queue(maxsize=seq_num)  

In [ ]:
# To save in memory the current ssd detection data
ssd_detection_data_dict = {}

In [ ]:
# Placeholder to capture the interm mse between the forecasted bbox data (LSTM) vs detected bbox data (SSD)
mse_1_forecast_eval   = []
mse_5_forecast_eval   = []
mse_10_forecast_eval  = []
mse_25_forecast_eval  = []
mse_50_forecast_eval  = []
mse_75_forecast_eval  = []
mse_100_forecast_eval = [] 

# Or use a dictionary object
# i.e., Keys = [1, 5, 10, 25, 75, 100], Values = [Arrays of MSEs for each new sky image and its forecasts]
mse_forecasts_eval = {} 

forecasting_counter = 0

#### Spawn the producer thread

In [ ]:
queue_of_inbound_sky_images.qsize() # Test<1>

In [ ]:
t = threading.Thread(target=sky_images_generator, args=(sky_camera_images_files, queue_of_inbound_sky_images,))
t.start()

In [ ]:
queue_of_inbound_sky_images.qsize() # Test<2,3, ..., k>

#### Get an image file from the queue in a FIFO format

In [ ]:
sample_image_filename = queue_of_inbound_sky_images.get()

In [ ]:
sample_image_filename

### Sample Image Detection Data

#### Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline

In [ ]:
sample_image_file = os.path.join(sky_camera_images_dir, sample_image_filename)

In [ ]:
results_detection_data = detection_data(sample_image_file)

In [ ]:
results_detection_data

In [ ]:
ssd_detection_data = post_process_detection_data(results_detection_data)

In [ ]:
ssd_detection_data_dict[list(ssd_detection_data.keys())[0]] = list(ssd_detection_data.values())[0]

In [ ]:
ssd_detection_data_dict

In [ ]:
def mse_evaluation_of_1_windowed_forecasted(candidate_key, next_frame_forecast):
    """ This assume that we access to n+1 future images access to validate the 
        LSTM forecasts of the next sky image ssd detecton data. """
    
    # Given a candidate next image (candidate_key), compute its SSD detection bbox data    
    print("\n:: Candidate Key-1: ", candidate_key)
    
    # Run SSD Detection
    results_detection_data = detection_data(candidate_key)
    ssd_detection_data = post_process_detection_data(results_detection_data)
    
    candidate_ssd_data = list(ssd_detection_data.values())[0]    
    print("\n:: candidate_ssd_data: ", candidate_ssd_data)
    
    padded_candidate_ssd_data = post_process_ssd_detection_data_for_forecasting(candidate_ssd_data, max_bboxes_vec_size)
    print("\n:: padded_candidate_ssd_data: ", padded_candidate_ssd_data)
    
    print("\n:: next_frame_forecast_data: ", next_frame_forecast)
    
    # Compute its mse aggr. value.
    mse_1_eval = mean_squared_error(padded_candidate_ssd_data, next_frame_forecast)
    
    return mse_1_eval

In [ ]:
def mse_evaluation_of_n_windowed_forecasted(candidate_key, next_n_frames_forecasts, forecast_window):
    """ This assume that we access to n+1 future images access to validate the 
        LSTM forecasts of the next n-forecast_window sky images ssd detecton data."""
    
    ssd_detection_data_list = []
    candidate_filenames = []
    candidate_key_filename = candidate_key.split("/")[-1]
    for i in range(len(sky_camera_images_files)):
        if sky_camera_images_files[i] == candidate_key_filename:
            candidate_filenames = sky_camera_images_files[i:i+forecast_window]
            break
    print("\n:: candidate_filenames-n: ", len(candidate_filenames), "\n", candidate_filenames)
    
    
    for image_filename in candidate_filenames:
        
        candidate_filepath = os.path.join(sky_camera_images_dir, image_filename)

        # Run SSD Detection
        results_detection_data = detection_data(candidate_filepath)
        ssd_detection_data = post_process_detection_data(results_detection_data)
        candidate_ssd_data = list(ssd_detection_data.values())[0]  
        padded_candidate_ssd_data = post_process_ssd_detection_data_for_forecasting(candidate_ssd_data, max_bboxes_vec_size)
        
        #print("\n:: padded_candidate_ssd_data: ", padded_candidate_ssd_data)
        
        ssd_detection_data_list.append(padded_candidate_ssd_data)
    
    ssd_detection_data_list = np.asarray(ssd_detection_data_list)
    
    print("\n\n:: ssd_detection_data_list: ", len(ssd_detection_data_list), ssd_detection_data_list, "\n")
    print("\n\n:: next_n_frames_forecasts: ", len(next_n_frames_forecasts), next_n_frames_forecasts, "\n")
    
    #print("\n\n:: next_n_frames_forecasts[0]: ", len(next_n_frames_forecasts[0]), next_n_frames_forecasts[0], "\n")
    #print(len(ssd_detection_data_list), len(next_n_frames_forecasts))
    print("\nCheck Mate: ", ssd_detection_data_list[0], next_n_frames_forecasts[0], "\n")

    # Compute its mse aggr. value.
    mse_n_eval_total = 0
    
    for i in range(len(ssd_detection_data_list)):
        mse_n_eval_total += mean_squared_error(ssd_detection_data_list[0], next_n_frames_forecasts[0])
    
    print("\nmse_n_eval_total: ", mse_n_eval_total, "\n")
    
    return float(mse_n_eval_total) / len(ssd_detection_data_list)


### Simulate inbound sky images

- Run SSD detection
- Create sequence of sky images to run forecast/predictions of future frame detection data and visualize them
- Run 24/7 or until stopped

In [ ]:
while(1):
    get_the_next_sky_image_filename = queue_of_inbound_sky_images.get() # FIFO mode
    print("\n:: Current sky image name: ", get_the_next_sky_image_filename)
    
    current_sky_image_filepath = os.path.join(sky_camera_images_dir, get_the_next_sky_image_filename)
    
    # Run SSD Detection
    results_detection_data = detection_data(current_sky_image_filepath)
    ssd_detection_data = post_process_detection_data(results_detection_data)
    
    # Append it into the dict of ssd detection data
    ssd_detection_data_dict[list(ssd_detection_data.keys())[0]] = list(ssd_detection_data.values())[0]
    candidate_key = list(ssd_detection_data.keys())[0]
    
    #(0)
    if queue_of_images_sequence_for_forecast_task.qsize() < seq_num:
        queue_of_images_sequence_for_forecast_task.put(candidate_key)
    
    #(1) If the forecast queue reaches seqNum, run the forecast model for the next 5, 10, 20, 40, 60, 80, 100 frames
    # And plot the frequency of predicted number of occlusion occurences in those predicted n-frames.
    if queue_of_images_sequence_for_forecast_task.qsize() == seq_num:
        print("\n\nTime to begin the forecast ...")
        
        # Pop the oldest entry from the queue and use it to extract seq_num detection data from the 
        # ssd_detection_data_dict
        popped_sky_image_filename = queue_of_images_sequence_for_forecast_task.get()
        print("Popped: ", popped_sky_image_filename)
        
        # Create the sequence of detection data to be used for the prediction/forecast task
        extracted_seq_num_detection_data = return_sequence_ssd_detection_data(popped_sky_image_filename, ssd_detection_data_dict, seq_num, max_bboxes_vec_size)
        #print("\nExtracted: ", extracted_seq_num_detection_data)
        print("\nExtracted data hidden.\n")
        
        # Next 1st Frame Forecast
        next_frame_forecast = forecast_inferrence(extracted_seq_num_detection_data, forecasting_model_name)
        # Later compare it with the candidtate_key (visually)
        print(":: Next 1st Frame Forecast: ", next_frame_forecast, "\n")
        
        num_occlusions_bool = forecast_report_on_occlusions_predictions(next_frame_forecast)
        print("\n:: Next 1st Frame Forecast -- Summary Report:\n\t >> In regards to the presence of an occlusion event in the next Frame, the forecast says: ", num_occlusions_bool, "\n")
    
        mse_1_eval = mse_evaluation_of_1_windowed_forecasted(candidate_key, next_frame_forecast)
        print("\n: mse_1_eval: ", mse_1_eval, "\n")
        
        # Saving each forecasted vs ssd detection mse for future reference
        mse_1_forecast_eval.append(mse_1_eval)
        
        ### Generic Forecast model for n-horizon window: Next [5, 10, 25, 50, 75, 100] Frames Forecast
        forecast_window = [5, 10, 25, 50, 75, 100]
        
        for i in forecast_window:
            print("\n\n:: Auto forecast the next <", i, "> frames.\n")
            next_n_frames_forecasts = n_windowed_forecast_inferrence(i, extracted_seq_num_detection_data, forecasting_model_name)
            
            # Forecast Report
            print("\n:: Summary of the ", i, " forecasts:\n", next_n_frames_forecasts, "\n")
            occlusion_bool_list = forecast_report_on_occlusions_predictions(next_n_frames_forecasts)
            print("\n:: Next", i, "Frames Forecast -- Summary Report:\n\t >> In regards to the presence of an occlusion event in the next Frames, the forecast says: ", occlusion_bool_list, "\n")
    
            mse_n_eval = mse_evaluation_of_n_windowed_forecasted(candidate_key, next_n_frames_forecasts, i)
            print("\n: mse_", i ,"_eval: ", mse_n_eval, "\n")

            # Saving each forecasted vs ssd detection mse for future reference
            if i == 5:
                mse_5_forecast_eval.append(mse_n_eval)
            if i == 10:
                mse_10_forecast_eval.append(mse_n_eval)
            if i == 25:
                mse_25_forecast_eval.append(mse_n_eval)
            if i == 50:
                mse_50_forecast_eval.append(mse_n_eval)
            if i == 75:
                mse_75_forecast_eval.append(mse_n_eval)
            if i == 100:
                mse_100_forecast_eval.append(mse_n_eval)    
    
            #break # Comment it out to run all the forecast_window targets.
        
        #(2) Put new entry in the forecast queue (0-1)
        queue_of_images_sequence_for_forecast_task.put(candidate_key)
        print("\n\n:: Newly Put: \n\t", candidate_key)
        
        forecasting_counter += 1 # Pre-Increment
        
        if forecasting_counter %100:
            print("\n\n\n:: Current forecasting_counter: ", forecasting_counter, "\n\n")
            
        if forecasting_counter > 10:
            print("\n:: mse_1_forecast_eval: ", mse_1_forecast_eval, "\n\n")
            print("\n:: mse_5_forecast_eval: ", mse_5_forecast_eval, "\n\n") 
            print("\n:: mse_10_forecast_eval: ", mse_10_forecast_eval, "\n\n")
            print("\n:: mse_25_forecast_eval: ", mse_25_forecast_eval, "\n\n")
            print("\n:: mse_50_forecast_eval: ", mse_50_forecast_eval, "\n\n")
            print("\n:: mse_75_forecast_eval: ", mse_75_forecast_eval, "\n\n")
            print("\n:: mse_100_forecast_eval: ", mse_100_forecast_eval, "\n\n")
            
            break
        
        #input("Enter to continue...") # To be commented out soon.

In [ ]:
print("Summary: ")
print("1-forecast - MSE(avg): ", sum(mse_1_forecast_eval)/len(mse_1_forecast_eval))
print("5-forecasts - MSE(avg): ", sum(mse_5_forecast_eval)/len(mse_5_forecast_eval))
print("10-forecasts - MSE(avg): ", sum(mse_10_forecast_eval)/len(mse_10_forecast_eval))
print("25-forecasts - MSE(avg): ", sum(mse_25_forecast_eval)/len(mse_25_forecast_eval))
print("50-forecasts - MSE(avg): ", sum(mse_50_forecast_eval)/len(mse_50_forecast_eval))
print("75-forecasts - MSE(avg): ", sum(mse_75_forecast_eval)/len(mse_75_forecast_eval))
print("100-forecasts - MSE(avg): ", sum(mse_100_forecast_eval)/len(mse_100_forecast_eval))

In [ ]:
print("Summary: ")
print("1-forecast - MSE(avg): ", sum(mse_1_forecast_eval), len(mse_1_forecast_eval))
print("5-forecasts - MSE(avg): ", sum(mse_5_forecast_eval), len(mse_5_forecast_eval))
print("10-forecasts - MSE(avg): ", sum(mse_10_forecast_eval), len(mse_10_forecast_eval))
print("25-forecasts - MSE(avg): ", sum(mse_25_forecast_eval), len(mse_25_forecast_eval))
print("50-forecasts - MSE(avg): ", sum(mse_50_forecast_eval), len(mse_50_forecast_eval))
print("75-forecasts - MSE(avg): ", sum(mse_75_forecast_eval), len(mse_75_forecast_eval))
print("100-forecasts - MSE(avg): ", sum(mse_100_forecast_eval), len(mse_100_forecast_eval))

In [ ]:
# Add a profiling module separately to estimate the range of areas of occlusions (LowerBound < X.Y < UpperBound)

In [ ]:
# Note: Uses old Tensorflow (not the 2.0), for local dev, uses conda research_tf venv.